In [1]:
%load_ext autoreload
%autoreload 2

%autosave 120
import sys
import pandas as pd
import glob
import numpy as np
import os
import random
from tqdm import tqdm
import json
from modules.utilities import are_bugs_from_tangled, has_bug, is_same_history, parse_hash_delimited_string
from modules.Untangler import Untangler, UntanglerOpenAI
from sklearn.metrics import (accuracy_score, f1_score, matthews_corrcoef, mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_error, roc_auc_score, root_mean_squared_error, precision_score, recall_score)
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay
from tabulate import tabulate
from sklearn.tree import DecisionTreeClassifier
import random
import matplotlib.pyplot as plt
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
random_seed = 42

Autosaving every 120 seconds


In [2]:
df = pd.read_csv("./data/Cleaned/ant.csv")
df = df[['SLOCStandard', 'Readability', 'SimpleReadability', 'NVAR',
       'NCOMP', 'Mcclure', 'McCabe', 'IndentSTD', 'MaximumBlockDepth',
       'totalFanOut', 'Length', 'MaintainabilityIndex', 'SATD', 'Parameters',
       'LocalVariables', "Decision", "Detection"]]
df

,SLOCStandard,Readability,SimpleReadability,NVAR,NCOMP,Mcclure,McCabe,IndentSTD,MaximumBlockDepth,totalFanOut,Length,MaintainabilityIndex,SATD,Parameters,LocalVariables,Decision,Detection
0,8#10#10#10#10,0.20740219950675964#0.07883955538272858#0.0776...,0.11547956518685411#0.11652725342266991#0.1139...,0#0#0#0#0,0#0#0#0#0,0#0#0#0#0,2#2#2#2#2,3.12#9.31#9.31#9.31#9.31,1#1#1#1#1,3#3#3#3#3,48.0#54.0#54.0#57.0#57.0,108.5035629272461#104.17417907714844#104.17417...,0#0#0#0#0,1#1#1#1#1,1#1#1#1#1,NotBuggy,NotBuggy
1,15,0.5179192423820496,0.6955678097377155,0,0,0,2,2.72,1,2,52.0,97.95835876464844,0,1,2,NotBuggy,NotBuggy
2,8#8#7,0.9325079917907715#0.2675686776638031#0.234025...,0.6681917303718937#0.5530645378073769#0.652247...,1#1#1,1#1#1,2#2#2,2#2#2,2.27#2.27#2.4,1#1#1,3#3#3,40.0#40.0#35.0,109.82251739501953#109.82251739501953#112.7543...,0#0#0,0#0#0,2#2#1,NotBuggy,NotBuggy
3,3,0.981898307800293,0.8516643188612856,0,0,0,1,1.89,0,0,20.0,130.30841064453125,0,0,0,NotBuggy,NotBuggy
4,3#3#4#4,0.9236592650413513#0.9276382327079773#0.931389...,0.8139134914218252#0.8207659381100223#0.893343...,0#0#0#0,0#0#0#0,0#0#0#0,1#1#1#1,1.89#1.89#12.95#3.32,0#0#0#0,0#0#0#0,17.0#17.0#20.0#20.0,131.43582153320312#131.43582153320312#125.7821...,0#0#0#0,0#0#0#0,0#0#0#0,NotBuggy,NotBuggy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9408,3#3,0.7683627605438232#0.5495423674583435,0.9309991825742876#0.8852181886941373,0#0,0#0,0#0,1#1,1.89#1.89,0#0,0#1,14.0#15.0,132.4454345703125#132.086669921875,0#0,0#0,0#0,NotBuggy,NotBuggy
9409,3,0.831268310546875,0.9436522995854255,0,0,0,1,1.89,0,0,13.0,132.9956512451172,0,0,0,NotBuggy,NotBuggy
9410,3,0.5723476409912109,0.8011991475439673,0,0,0,1,1.89,0,2,19.0,130.70933532714844,0,0,0,NotBuggy,NotBuggy
9411,8#9#8#3,0.3672192096710205#0.3538154363632202#0.381029...,0.9059220611853999#0.898949087330709#0.7904578...,0#0#0#0,0#0#0#0,0#0#0#0,1#1#1#1,14.22#18.51#19.46#1.89,1#1#1#0,2#2#2#1,36.0#37.0#38.0#21.0,110.67463684082031#108.5498275756836#110.31923...,0#0#0#0,1#1#1#1,0#0#0#0,NotBuggy,NotBuggy


In [3]:
X = df.drop(["Decision","Detection"], axis=1)
X = X.map(lambda x: x.split("#")[0])
X = X.astype(float)

enc1 = LabelEncoder()
Y_Prev = enc1.fit_transform(df["Decision"])

enc2 = LabelEncoder()
Y = enc2.fit_transform(df["Detection"])

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score


cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_seed)

acc = []
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []
precision_weighted = []
recall_weighted = []
roc = []
avg_precision = []

for train, test in tqdm(cv.split(X, Y)):
    x_train = X.iloc[train]
    x_test = X.iloc[test]

    y_train = Y[train]
    y_test = Y[test]

    rus = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
    x_train, y_train = rus.fit_resample(x_train, y_train)
    
    model = RandomForestClassifier(random_state=random_seed)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    
    acc.append(accuracy_score(y_test, y_pred))

    precision_micro.append(precision_score(y_test, y_pred, average="micro"))
    recall_micro.append(recall_score(y_test, y_pred, average="micro"))

    precision_macro.append(precision_score(y_test, y_pred, average="macro"))
    recall_macro.append(recall_score(y_test, y_pred, average="macro"))

    precision_weighted.append(precision_score(y_test, y_pred, average="weighted"))
    recall_weighted.append(recall_score(y_test, y_pred, average="weighted"))

    roc.append(roc_auc_score(y_test, y_pred))
    avg_precision.append(average_precision_score(y_test, y_pred))

print(f"Accuracy: {np.mean(acc):.4f} ± {np.std(acc):.4f}")
print(f"Avg Precision: {np.mean(avg_precision):.4f} ± {np.std(avg_precision):.4f}")
print(f"Micro Precision: {np.mean(precision_micro):.4f} ± {np.std(precision_micro):.4f}")
print(f"Micro Recall: {np.mean(recall_micro):.4f} ± {np.std(recall_micro):.4f}")
print(f"Macro Precision: {np.mean(precision_macro):.4f} ± {np.std(precision_macro):.4f}")
print(f"Macro Recall: {np.mean(recall_macro):.4f} ± {np.std(recall_macro):.4f}")
print(f"Weighted Precision: {np.mean(precision_weighted):.4f} ± {np.std(precision_weighted):.4f}")
print(f"Weighted Recall: {np.mean(recall_weighted):.4f} ± {np.std(recall_weighted):.4f}")
print(f"ROC: {np.mean(roc):.4f} ± {np.std(roc):.4f}")

10it [00:02,  3.93it/s]

Accuracy: 0.8325 ± 0.0219
Avg Precision: 0.9872 ± 0.0014
Micro Precision: 0.8325 ± 0.0219
Micro Recall: 0.8325 ± 0.0219
Macro Precision: 0.5304 ± 0.0040
Macro Recall: 0.7038 ± 0.0332
Weighted Precision: 0.9692 ± 0.0019
Weighted Recall: 0.8325 ± 0.0219
ROC: 0.7038 ± 0.0332


In [10]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_seed)

acc = []
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []
precision_weighted = []
recall_weighted = []

for train, test in tqdm(cv.split(X, Y_Prev)):
    x_train = X.iloc[train]
    x_test = X.iloc[test]

    y_train = Y_Prev[train]
    y_test = Y_Prev[test]

    rus = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
    x_train, y_train = rus.fit_resample(x_train, y_train)
    
    model = RandomForestClassifier(random_state=random_seed)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    
    acc.append(accuracy_score(y_test, y_pred))

    precision_micro.append(precision_score(y_test, y_pred, average="micro"))
    recall_micro.append(recall_score(y_test, y_pred, average="micro"))

    precision_macro.append(precision_score(y_test, y_pred, average="macro"))
    recall_macro.append(recall_score(y_test, y_pred, average="macro"))

    precision_weighted.append(precision_score(y_test, y_pred, average="weighted"))
    recall_weighted.append(recall_score(y_test, y_pred, average="weighted"))

    roc.append(roc_auc_score(y_test, y_pred))
    avg_precision.append(average_precision_score(y_test, y_pred))

print(f"Accuracy: {np.mean(acc):.4f} ± {np.std(acc):.4f}")
print(f"Avg Precision: {np.mean(avg_precision):.4f} ± {np.std(avg_precision):.4f}")
print(f"Micro Precision: {np.mean(precision_micro):.4f} ± {np.std(precision_micro):.4f}")
print(f"Micro Recall: {np.mean(recall_micro):.4f} ± {np.std(recall_micro):.4f}")
print(f"Macro Precision: {np.mean(precision_macro):.4f} ± {np.std(precision_macro):.4f}")
print(f"Macro Recall: {np.mean(recall_macro):.4f} ± {np.std(recall_macro):.4f}")
print(f"Weighted Precision: {np.mean(precision_weighted):.4f} ± {np.std(precision_weighted):.4f}")
print(f"Weighted Recall: {np.mean(recall_weighted):.4f} ± {np.std(recall_weighted):.4f}")
print(f"ROC: {np.mean(roc):.4f} ± {np.std(roc):.4f}")

10it [00:02,  4.18it/s]

Accuracy: 0.8304 ± 0.0163
Avg Precision: 0.9854 ± 0.0026
Micro Precision: 0.8304 ± 0.0163
Micro Recall: 0.8304 ± 0.0163
Macro Precision: 0.5429 ± 0.0090
Macro Recall: 0.7194 ± 0.0383
Weighted Precision: 0.9606 ± 0.0029
Weighted Recall: 0.8304 ± 0.0163
ROC: 0.7116 ± 0.0367
